# Importing VTK data to be displayed as Mesh

Additional requirements for this example: `vtk`

We will also see how to do some processing with `vtk` and visualize its results.

In [ ]:
import k3d
import os
import vtk
try:
    from urllib.request import urlopen
except ImportError:
    import urllib2, contextlib
    urlopen = lambda url: contextlib.closing(urllib2.urlopen(url))

filename = 'cow.vtp'
source = "https://raw.githubusercontent.com/naucoin/VTKData/master/Data/cow.vtp"

model_matrix = (
     1.0,  0.0, 0.0, 0.0,
     0.0,  0.0, 1.0, 0.0,
     0.0,  1.0, 0.0, 0.0,
     0.0,  0.0, 0.0, 1.0
)

if not os.path.isfile(filename):
    print('Downloading model')
    with urlopen(source) as response, open(filename, 'w') as f:
        f.write(response.read().decode('utf8'))
    
reader = vtk.vtkXMLPolyDataReader()
reader.SetFileName(filename)
reader.Update()

plot = k3d.plot()
cow3d = k3d.vtk_poly_data(reader.GetOutput(), color=0xff0000, model_matrix=model_matrix)
plot += cow3d
plot.display()

In [ ]:
plane=vtk.vtkPlane()
plane.SetOrigin(0,0,0)
plane.SetNormal(1,1,0)
 
cutter=vtk.vtkCutter()
cutter.SetCutFunction(plane)
cutter.SetInputConnection(reader.GetOutputPort())
cutter.Update()

FeatureEdges = vtk.vtkFeatureEdges()
FeatureEdges.SetInputConnection(cutter.GetOutputPort())
FeatureEdges.BoundaryEdgesOn()
FeatureEdges.FeatureEdgesOff()
FeatureEdges.NonManifoldEdgesOff()
FeatureEdges.ManifoldEdgesOff()
FeatureEdges.Update()

cutStrips = vtk.vtkStripper() ; #Forms loops (closed polylines) from cutter
cutStrips.SetInputConnection(cutter.GetOutputPort())
cutStrips.Update()

cutPoly = vtk.vtkPolyData() ; #This trick defines polygons as polyline loop
cutPoly.SetPoints((cutStrips.GetOutput()).GetPoints())
cutPoly.SetPolys((cutStrips.GetOutput()).GetLines())

In [ ]:
model_matrix = (
     1.0,  0.0, 0.0, 0.0,
     0.0,  0.0, 1.0, -5.0,
     0.0,  1.0, 0.0, 0.0,
     0.0,  0.0, 0.0, 1.0
)
plot += k3d.vtk_poly_data(cutPoly, color=0x0000ff,  model_matrix=model_matrix)